In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

PRESET = os.environ.setdefault("PRESET", "millionaires")
SELF_PARTY = os.environ.setdefault("SELF_PARTY", "sim")

In [3]:
import ray
from opentelemetry import trace
from opentelemetry.sdk.environment_variables import OTEL_SERVICE_NAME

from secretnote.instrumentation import Profiler
from secretnote.instrumentation.sdk import (
    setup_tracing,
    setup_debug_exporter,
    inherit_tracing_context,
)
from secretnote.instrumentation.rules import create_default_rules

ray.shutdown()

create_default_rules()

os.environ[OTEL_SERVICE_NAME] = f"{PRESET}:{SELF_PARTY}"

setup_tracing()
setup_debug_exporter()

with trace.get_tracer(__name__).start_as_current_span(
    "sim_trace",
    context=inherit_tracing_context(),
):
    with Profiler() as profiler:
        with open(f"presets/{PRESET}/_world.py") as f:
            _world = compile(f.read(), f.name, "exec")
        with open(f"presets/{PRESET}/_algorithm.py") as f:
            _algorithm = compile(f.read(), f.name, "exec")
        exec(_world, globals())
        exec(_algorithm, globals())

2023-10-30 13:46:44,668	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-10-30 13:46:47,825	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.
(_run pid=34382) INFO:jax._src.xla_bridge:Unable to initialize backen

(SPURuntime pid=34402) 2023-10-30 13:46:49.998 [info] [default_brpc_retry_policy.cc:DoRetry:52] socket error, sleep=1000000us and retry
(SPURuntime pid=34402) 2023-10-30 13:46:50.999 [info] [default_brpc_retry_policy.cc:LogHttpDetail:33] cntl ErrorCode '64', http status code '200', response header '', error msg '[E61]Fail to connect Socket{id=0 addr=127.0.0.1:32768} (0x0x1400c8000): Connection refused [R1][E64]Not connected to 127.0.0.1:32768 yet, server_id=0'
(SPURuntime pid=34402) 2023-10-30 13:46:50.999 [info] [default_brpc_retry_policy.cc:DoRetry:75] aggressive retry, sleep=1000000us and retry
(SPURuntime pid=34402) 2023-10-30 13:46:51.999 [info] [default_brpc_retry_policy.cc:LogHttpDetail:33] cntl ErrorCode '64', http status code '200', response header '', error msg '[E61]Fail to connect Socket{id=0 addr=127.0.0.1:32768} (0x0x1400c8000): Connection refused [R1][E64]Not connected to 127.0.0.1:32768 yet, server_id=0 [R2][E64]Not connected to 127.0.0.1:32768 yet, server_id=0'
(SPURun

In [72]:
from secretnote.display.app import visualize_run
visualize_run(profiler)